# Clean Tweets Indonesia

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
#from nltk.stem.wordnet import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

In [6]:
#DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
#data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding =DATASET_ENCODING, names=DATASET_COLUMNS)
data = pd.read_csv('tweets_662k.csv', encoding =DATASET_ENCODING, dtype={'text': "string"})
#data.drop(data.iloc[:, 3:13], inplace = True, axis = 1) 
data['text']=data['text'].apply(str)
X = data.iloc[:,[2]]
#Y = data.iloc[:,0]
#Y[Y == 4] = 1

In [ ]:
#data.head()
X['text'].head()

In [ ]:
# Text-preprocessing

# Missing Values
num_missing_desc = data.isnull().sum()[2]    # No. of values with msising descriptions
print('Number of missing values: ' + str(num_missing_desc))
data = data.dropna()

TAG_CLEANING_RE = "@\S+"
# Remove @tags
X['text'] = X['text'].map(lambda x: re.sub(TAG_CLEANING_RE, ' ', x))

# Smart lowercase
X['text'] = X['text'].map(lambda x: x.lower())

# Remove numbers
X['text'] = X['text'].map(lambda x: re.sub(r'\d+', ' ', x))

# Remove links
TEXT_CLEANING_RE = "https?:\S+|http?:\S|[^A-Za-z0-9]+"
X['text'] = X['text'].map(lambda x: re.sub(TEXT_CLEANING_RE, ' ', x))

# Remove Punctuation
X['text']  = X['text'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))

# Remove white spaces
X['text'] = X['text'].map(lambda x: x.strip())

# Tokenize into words
X['text'] = X['text'].map(lambda x: word_tokenize(x))
 
# Remove non alphabetic tokens
X['text'] = X['text'].map(lambda x: [word for word in x if word.isalpha()])

# Filter out stop words
stop_words = set(stopwords.words('indonesian'))
X['text'] = X['text'].map(lambda x: [w for w in x if not w in stop_words])
    
# Word Lemmatization
#lem = WordNetLemmatizer()
#X['text'] = X['text'].map(lambda x: [lem.lemmatize(word,"v") for word in x])


In [ ]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in X['text']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

X['text'] = X['text'].swifter.apply(get_stemmed_term)
#print(TWEET_DATA['tweet_tokens_stemmed'])

# Turn lists back to string
X['text'] = X['text'].map(lambda x: ' '.join(x))

In [ ]:
X.head()

In [7]:
X.to_csv("Clean_Tweets_Indonesia.csv")